<a href="https://colab.research.google.com/github/BhardwajArjit/Causality-Informed-Segmentation/blob/main/Semantic_Segmentation_UNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF

In [2]:
class DoubleConv(nn.Module):
  def __init__(self, in_channels, out_channels):
    super().__init__()

    self.conv = nn.Sequential(
        nn.Conv2d(in_channels=in_channels,
                  out_channels=out_channels,
                  kernel_size=3,
                  stride=1,
                  padding=1,
                  bias=False),
        nn.BatchNorm2d(num_features=out_channels),
        nn.ReLU(inplace=True),

        nn.Conv2d(in_channels=out_channels,
                  out_channels=out_channels,
                  kernel_size=3,
                  stride=1,
                  padding=1,
                  bias=False),
        nn.BatchNorm2d(num_features=out_channels),
        nn.ReLU(inplace=True)
    )

  def forward(self, x):
    return self.conv(x)

In [3]:
class UNet(nn.Module):
  def __init__(self,
               in_channels=3,
               out_channels=1,
               features=[64, 128, 256, 512]):
    super().__init__()

    self.ups = nn.ModuleList()
    self.downs = nn.ModuleList()
    self.pool = nn.MaxPool2d(kernel_size=2,
                             stride=2)

    for feature in features:
      self.downs.append(DoubleConv(in_channels=in_channels,
                                   out_channels=feature))
      in_channels=feature


    for feature in reversed(features):
      self.ups.append(nn.ConvTranspose2d(in_channels=feature*2,
                                         out_channels=feature,
                                         kernel_size=2,
                                         stride=2))
      self.ups.append(DoubleConv(in_channels=feature*2,
                                 out_channels=feature))

    self.bottleneck = DoubleConv(features[-1], features[-1] * 2)
    self.final_conv = nn.Conv2d(in_channels=features[0],
                                out_channels=out_channels,
                                kernel_size=1)

  def forward(self, x):
    skip_connections = []

    for down in self.downs:
      x = down(x)
      skip_connections.append(x)
      x = self.pool(x)

    x = self.bottleneck(x)
    skip_connections = skip_connections[::-1]

    for idx in range(0, len(self.ups), 2):
      x = self.ups[idx](x)
      skip_connection = skip_connections[idx // 2]

      if x.shape != skip_connection.shape:
        x = TF.resize(x, size=skip_connection.shape[2:])

      concat_skip = torch.cat((skip_connection, x), dim=1)
      x = self.ups[idx + 1](concat_skip)

    return self.final_conv(x)

In [4]:
def test():
  x = torch.randn((3, 1, 160, 160))
  model = UNet(in_channels=1, out_channels=1)
  preds = model(x)
  print(preds.shape)
  print(x.shape)

  assert preds.shape == x.shape

In [6]:
test()

torch.Size([3, 1, 160, 160])
torch.Size([3, 1, 160, 160])
